In [9]:
import sys
sys.path.append('../')
import env

In [10]:
import pandas as pd
import numpy as np

In [11]:
READ_FILE = '2D'
WRITE_FILE = 'Timeflow'

In [13]:
times_list = [str(3600 * (i + 1)) for i in range(6)]

In [5]:
csv_array = {}
def main(args, array):
    df = pd.read_csv(env.get_full_path(READ_FILE, args),
                     encoding='Shift_JISx0213')
    array[args.dir][args.ratio][args.seed][args.csv] = df.copy()
    
env.for_default_init(main, csv_array)

In [6]:
timeflow_array = {}
def create_timeflow(args, array):
    df_base = pd.DataFrame(columns=times_list)
    df = csv_array[args.dir][args.ratio][args.seed][args.csv].copy()

    
    group = df.groupby('area')
    for _group in group:
        # エリア番号とグルーピングした中身に分ける
        area = int(_group[0])
        df_group = _group[1].sort_values('time')
        
        # 人数だけ抽出し時系列順にDataFrameに格納
        df_group = df_group.loc[:, 'people'].reset_index(drop=True)
        
        # 要素が1つだけだとseriesになってしまうので
        df_group = pd.DataFrame(df_group).T
        
        # coumnsを設定して結合
        df_group.columns = times_list
        df_base = pd.concat([df_base, df_group])
        
    # indexがpeopleになるのでリセット
    df_base.reset_index(drop=True, inplace=True)
    array[args.dir][args.ratio][args.seed][args.csv] = df_base.copy()
    
env.for_default_init(create_timeflow, timeflow_array)

In [7]:
# def output_file(args):
#     df = timeflow_array[args.dir][args.ratio][args.seed][args.csv].copy()
#     df.to_csv(env.get_full_path(WRITE_FILE, args))
    
# env.for_default(output_file)